In [1]:
import numpy as np
import pandas as pd
import ast
from lightweight_charts import Chart
from lightweight_charts.widgets import StreamlitChart
import json
import streamlit

In [2]:
df = pd.read_csv('F:\ML_projects\Internship1\data\TSLA_data - Sheet1.csv', parse_dates=['timestamp'])

In [3]:
df.head(5)

,timestamp,direction,Support,Resistance,open,high,low,close,volume
0,2022-08-25,SHORT,"[840, 880]","[900, 895, 940]",302.542,302.807,291.556,295.897,6685.7261
1,2022-08-26,SHORT,"[877, 880, 840, 835, 845]","[900, 908, 980.5, 940]",297.297,301.956,287.376,288.017,508.5590
2,2022-08-27,SHORT,[290],"[300, 322, 323, 324, 422]",297.297,301.956,287.376,288.017,508.5590
3,2022-08-28,LONG,"[285, 286, 84]","[290, 292, 295]",297.297,301.956,287.376,288.017,508.5590
4,2022-08-31,SHORT,"[280, 247, 150, 185]","[287, 288, 292]",280.497,280.837,271.776,275.337,94.2867


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   665 non-null    datetime64[ns]
 1   direction   335 non-null    object        
 2   Support     665 non-null    object        
 3   Resistance  665 non-null    object        
 4   open        665 non-null    float64       
 5   high        665 non-null    float64       
 6   low         665 non-null    float64       
 7   close       665 non-null    float64       
 8   volume      665 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 46.9+ KB


In [5]:
df.isna().sum()

timestamp       0
direction     330
Support         0
Resistance      0
open            0
high            0
low             0
close           0
volume          0
dtype: int64

In [6]:
df[["open","high","low","close","volume"]].describe()

,open,high,low,close,volume
count,665.000000,665.000000,665.000000,665.000000,665.000000
mean,233.765603,239.119302,228.496779,233.849032,100.987451
std,65.874333,67.713161,64.107825,65.773434,258.916828
min,102.916000,111.717000,101.200000,110.336000,32.727600
25%,184.967000,186.746000,181.257000,184.956000,67.935600
50%,224.057000,227.746000,218.197000,220.897000,86.149200
75%,259.126000,263.587000,253.096000,260.326000,107.366400
max,466.117000,488.306000,450.996000,460.396000,6685.726100


In [7]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """
    - Parse Support/Resistance string lists into Python lists safely.
    - Compute support/resistance bounds.
    - Use the provided `direction` column (preserving missing values).
    - Compute marker positions based on direction.
    """

    def parse_list(s: str) -> list:
        s = str(s).strip()
        try:
            lst = ast.literal_eval(s)
            return lst if isinstance(lst, list) else []
        except (ValueError, SyntaxError):
            return []

    df["Support_list"]    = df["Support"].apply(parse_list)
    df["Resistance_list"] = df["Resistance"].apply(parse_list)
    df["sup_low"]  = df["Support_list"].apply(lambda x: min(x) if x else None)
    df["sup_high"] = df["Support_list"].apply(lambda x: max(x) if x else None)
    df["res_low"]  = df["Resistance_list"].apply(lambda x: min(x) if x else None)
    df["res_high"] = df["Resistance_list"].apply(lambda x: max(x) if x else None)
    df["direction"] = df["direction"].str.upper().replace({"SHORT": "SHORT", "LONG": "LONG"})


    def marker_price(r):
        if r["direction"] == "LONG":
            return r["low"] * 0.995 if pd.notnull(r["low"]) else None
        if r["direction"] == "SHORT":
            return r["high"] * 1.005 if pd.notnull(r["high"]) else None
        return None

    df["marker_price"] = df.apply(marker_price, axis=1)

    return df

In [8]:
preprocess(df)
df.head(5)

,timestamp,direction,Support,Resistance,open,high,low,close,volume,Support_list,Resistance_list,sup_low,sup_high,res_low,res_high,marker_price
0,2022-08-25,SHORT,"[840, 880]","[900, 895, 940]",302.542,302.807,291.556,295.897,6685.7261,"[840, 880]","[900, 895, 940]",840.0,880.0,895.0,940.0,304.321035
1,2022-08-26,SHORT,"[877, 880, 840, 835, 845]","[900, 908, 980.5, 940]",297.297,301.956,287.376,288.017,508.5590,"[877, 880, 840, 835, 845]","[900, 908, 980.5, 940]",835.0,880.0,900.0,980.5,303.465780
2,2022-08-27,SHORT,[290],"[300, 322, 323, 324, 422]",297.297,301.956,287.376,288.017,508.5590,[290],"[300, 322, 323, 324, 422]",290.0,290.0,300.0,422.0,303.465780
3,2022-08-28,LONG,"[285, 286, 84]","[290, 292, 295]",297.297,301.956,287.376,288.017,508.5590,"[285, 286, 84]","[290, 292, 295]",84.0,286.0,290.0,295.0,285.939120
4,2022-08-31,SHORT,"[280, 247, 150, 185]","[287, 288, 292]",280.497,280.837,271.776,275.337,94.2867,"[280, 247, 150, 185]","[287, 288, 292]",150.0,280.0,287.0,292.0,282.241185


In [9]:
from lightweight_charts import Chart
from IPython.display import HTML

def render_chart(df):
    chart = Chart(toolbox=True)  # enables drawing features
    
    df = df.copy()
    df["time"] = df["timestamp"].dt.strftime("%Y-%m-%d")
    chart.set(df[["time", "open", "high", "low", "close"]])

    # --- add support & resistance rectangles ---
    for _, row in df.iterrows():
        if pd.notnull(row["sup_low"]) and pd.notnull(row["sup_high"]):
            chart.rectangle(
                price_min=row["sup_low"],
                price_max=row["sup_high"],
                time_start=row["time"],
                time_end=row["time"],
                color="rgba(0,255,0,0.2)",
                border_color="rgba(0,255,0,0.5)",
                extend_right=True
            )

        if pd.notnull(row["res_low"]) and pd.notnull(row["res_high"]):
            chart.rectangle(
                price_min=row["res_low"],
                price_max=row["res_high"],
                time_start=row["time"],
                time_end=row["time"],
                color="rgba(255,0,0,0.2)",
                border_color="rgba(255,0,0,0.5)",
                extend_right=True
            )

    # --- add direction markers as text ---
    for _, row in df.iterrows():
        if row["direction"] == "LONG":
            chart.text(
                text="↑",
                price=row["marker_price"],
                time=row["time"],
                color="green",
                size=18,
                align="bottom"
            )
        elif row["direction"] == "SHORT":
            chart.text(
                text="↓",
                price=row["marker_price"],
                time=row["time"],
                color="red",
                size=18,
                align="top"
            )
        else:
            chart.text(
                text="⚫",
                price=row["close"],  # center of candle
                time=row["time"],
                color="yellow",
                size=10,
                align="center"
            )

    return HTML(chart.to_html())

In [10]:
render_chart(df)

AttributeError: 'Chart' object has no attribute 'rectangle'

In [ ]:
import lightweight_charts
print(lightweight_charts.__file__)
print(lightweight_charts.__version__)